In [ ]:
%pip install nba_api

In [ ]:
"""


api link:       https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/leaguegamefinder.md
                https://github.com/swar/nba_api/tree/master/docs/nba_api
code base link: https://github.com/liannewriting/data-science-project-from-scratch-youtube-nba/blob/main/main.py
                https://youtu.be/cSnM2PHVqYs

"""

In [1]:
# import libraries

from nba_api.stats.endpoints import leaguegamefinder
import datetime as dt
import pandas as pd
import numpy as np

In [2]:
league_start= dt.date(2022,10,17)   #league start
league_end= dt.date.today()

# to get a stupid MM/DD/YYYY format to push the parameter through
league_start_str = league_start.strftime("%m/%d/%Y")
league_end_str = league_end.strftime("%m/%d/%Y")




In [3]:
gamefinder = leaguegamefinder.LeagueGameFinder(date_from_nullable=league_start_str, # start date
                                               date_to_nullable=league_end_str,     # end date
                                               league_id_nullable='00',
                                               )
# would be for specific date: date_from_nullable=game_date_str
games = gamefinder.get_data_frames()[0]
games.head(20)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1610612746,LAC,LA Clippers,0022200036,2022-10-22,LAC @ SAC,W,241,111,...,0.909,5,35,40,24,8,3,13,21,2.0
1,22022,1610612758,SAC,Sacramento Kings,0022200036,2022-10-22,SAC vs. LAC,L,239,109,...,0.577,9,30,39,24,5,5,12,21,-2.0
2,22022,1610612749,MIL,Milwaukee Bucks,0022200033,2022-10-22,MIL vs. HOU,W,241,125,...,0.619,8,39,47,29,6,10,16,22,20.0
3,22022,1610612745,HOU,Houston Rockets,0022200033,2022-10-22,HOU @ MIL,L,242,105,...,0.773,15,32,47,20,9,6,12,22,-20.0
4,22022,1610612748,MIA,Miami Heat,0022200031,2022-10-22,MIA vs. TOR,W,240,112,...,0.926,6,38,44,28,2,2,15,25,3.0
5,22022,1610612761,TOR,Toronto Raptors,0022200031,2022-10-22,TOR @ MIA,L,241,109,...,0.750,8,29,37,24,9,5,6,25,-3.0
6,22022,1610612742,DAL,Dallas Mavericks,0022200034,2022-10-22,DAL vs. MEM,W,238,137,...,0.833,15,37,52,23,7,7,15,23,41.0
7,22022,1610612763,MEM,Memphis Grizzlies,0022200034,2022-10-22,MEM @ DAL,L,239,96,...,0.792,6,23,29,20,4,8,17,29,-41.0
8,22022,1610612743,DEN,Denver Nuggets,0022200035,2022-10-22,DEN vs. OKC,W,239,122,...,0.581,9,37,46,33,5,6,18,21,5.0
9,22022,1610612760,OKC,Oklahoma City Thunder,0022200035,2022-10-22,OKC @ DEN,L,239,117,...,0.783,10,37,47,29,10,5,12,27,-5.0


In [4]:
games.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [5]:
# Add additional information in separate columns from MATCHDAY

# Add a column about an opponent - always in matchup

opponent = games['MATCHUP'].str[-3:]
games['OPPONENT'] = opponent

# Is it a home or away game? > first set all to home then change those having @ to away
games['IS_HOME'] = 1
games.loc[games['MATCHUP'].str.contains('@'), 'IS_HOME'] = 0

# Add actual countable Wins
games['IS_WIN'] = 1
games.loc[games['WL'] == 'L', 'IS_WIN'] = 0

In [6]:
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,OPPONENT,IS_HOME,IS_WIN
0,22022,1610612746,LAC,LA Clippers,0022200036,2022-10-22,LAC @ SAC,W,241,111,...,40,24,8,3,13,21,2.0,SAC,0,1
1,22022,1610612758,SAC,Sacramento Kings,0022200036,2022-10-22,SAC vs. LAC,L,239,109,...,39,24,5,5,12,21,-2.0,LAC,1,0
2,22022,1610612749,MIL,Milwaukee Bucks,0022200033,2022-10-22,MIL vs. HOU,W,241,125,...,47,29,6,10,16,22,20.0,HOU,1,1
3,22022,1610612745,HOU,Houston Rockets,0022200033,2022-10-22,HOU @ MIL,L,242,105,...,47,20,9,6,12,22,-20.0,MIL,0,0
4,22022,1610612748,MIA,Miami Heat,0022200031,2022-10-22,MIA vs. TOR,W,240,112,...,44,28,2,2,15,25,3.0,TOR,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,22022,1610612758,SAC,Sacramento Kings,0022200014,2022-10-19,SAC vs. POR,L,241,108,...,41,27,8,5,15,25,-7.0,POR,1,0
68,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,...,48,31,11,4,18,23,14.0,LAL,1,1
69,22022,1610612747,LAL,Los Angeles Lakers,0022200002,2022-10-18,LAL @ GSW,L,241,109,...,48,23,12,4,21,18,-14.0,GSW,0,0
70,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,239,117,...,31,16,8,3,14,25,-9.0,BOS,0,0


In [7]:
teams = games.drop_duplicates(['TEAM_ID','TEAM_ABBREVIATION','TEAM_NAME'])[['TEAM_ID','TEAM_ABBREVIATION','TEAM_NAME']]
teams

,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME
0,1610612746,LAC,LA Clippers
1,1610612758,SAC,Sacramento Kings
2,1610612749,MIL,Milwaukee Bucks
3,1610612745,HOU,Houston Rockets
4,1610612748,MIA,Miami Heat
5,1610612761,TOR,Toronto Raptors
6,1610612742,DAL,Dallas Mavericks
7,1610612763,MEM,Memphis Grizzlies
8,1610612743,DEN,Denver Nuggets
9,1610612760,OKC,Oklahoma City Thunder


In [8]:
# Actually import the team selection (probably I want to move this the very start later on)
url ='https://raw.githubusercontent.com/vedranbrozovic/NBA_fantasy-tams/main/Team_selection.txt'
team_selection = pd.read_csv(url,sep='\t')
team_selection


,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,TEAM_OWNER,TEAM_PICK
0,1610612745,HOU,Houston Rockets,Miran,10
1,1610612761,TOR,Toronto Raptors,Miran,2
2,1610612763,MEM,Memphis Grizzlies,Miran,3
3,1610612765,DET,Detroit Pistons,Miran,8
4,1610612755,PHI,Philadelphia 76ers,Miran,1
5,1610612754,IND,Indiana Pacers,Miran,9
6,1610612737,ATL,Atlanta Hawks,Miran,6
7,1610612750,MIN,Minnesota Timberwolves,Miran,5
8,1610612756,PHX,Phoenix Suns,Miran,4
9,1610612752,NYK,New York Knicks,Miran,7


In [9]:
# Add info on the team owner & pick number to the games dataframe
team_selection_df = team_selection[['TEAM_ID','TEAM_OWNER','TEAM_PICK']]

games = pd.merge(games, team_selection_df, on=["TEAM_ID"])



,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,BLK,TOV,PF,PLUS_MINUS,OPPONENT,IS_HOME,IS_WIN,TEAM_OWNER,TEAM_PICK,OPPOSING_OWNER
0,22022,1610612746,LAC,LA Clippers,0022200036,2022-10-22,LAC @ SAC,W,241,111,...,3,13,21,2.0,SAC,0,1,Tata,3,Tata
1,22022,1610612757,POR,Portland Trail Blazers,0022200014,2022-10-19,POR @ SAC,W,239,115,...,2,11,17,7.0,SAC,0,1,Vedran,5,Tata
2,22022,1610612746,LAC,LA Clippers,0022200016,2022-10-20,LAC @ LAL,W,239,103,...,7,22,24,6.0,LAL,0,1,Tata,3,Vedran
3,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,...,4,18,23,14.0,LAL,1,1,Tata,2,Vedran
4,22022,1610612758,SAC,Sacramento Kings,0022200036,2022-10-22,SAC vs. LAC,L,239,109,...,5,12,21,-2.0,LAC,1,0,Tata,10,Tata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,22022,1610612752,NYK,New York Knicks,0022200023,2022-10-21,NYK vs. DET,W,239,130,...,6,12,26,24.0,DET,1,1,Miran,7,Miran
68,22022,1610612741,CHI,Chicago Bulls,0022200019,2022-10-21,CHI @ WAS,L,240,100,...,3,13,15,-2.0,WAS,0,0,Vedran,7,Tata
69,22022,1610612754,IND,Indiana Pacers,0022200004,2022-10-19,IND vs. WAS,L,241,107,...,5,15,20,-7.0,WAS,1,0,Miran,9,Tata
70,22022,1610612751,BKN,Brooklyn Nets,0022200006,2022-10-19,BKN vs. NOP,L,242,108,...,10,16,25,-22.0,NOP,1,0,Vedran,4,Tata


In [ ]:
# Add info on the owner of opposing team
opp_selection = team_selection[['TEAM_OWNER','TEAM_ABBREVIATION']]
opp_selection.columns = ['OPPOSING_OWNER','OPPONENT']
games = pd.merge(games, opp_selection ,on=["OPPONENT"])
games

In [19]:
# Add column to know if the game is against own team
games['IS_NOT_AGAINST_OWN'] = 1
games.loc[(games['TEAM_OWNER'] == games['OPPOSING_OWNER']),'IS_NOT_AGAINST_OWN']=0

games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,PF,PLUS_MINUS,OPPONENT,IS_HOME,IS_WIN,TEAM_OWNER,TEAM_PICK,OPPOSING_OWNER,IS_AGAINST_OWN,IS_NOT_AGAINST_OWN
0,22022,1610612746,LAC,LA Clippers,0022200036,2022-10-22,LAC @ SAC,W,241,111,...,21,2.0,SAC,0,1,Tata,3,Tata,1,0
1,22022,1610612757,POR,Portland Trail Blazers,0022200014,2022-10-19,POR @ SAC,W,239,115,...,17,7.0,SAC,0,1,Vedran,5,Tata,0,1
2,22022,1610612746,LAC,LA Clippers,0022200016,2022-10-20,LAC @ LAL,W,239,103,...,24,6.0,LAL,0,1,Tata,3,Vedran,0,1
3,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,...,23,14.0,LAL,1,1,Tata,2,Vedran,0,1
4,22022,1610612758,SAC,Sacramento Kings,0022200036,2022-10-22,SAC vs. LAC,L,239,109,...,21,-2.0,LAC,1,0,Tata,10,Tata,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,22022,1610612752,NYK,New York Knicks,0022200023,2022-10-21,NYK vs. DET,W,239,130,...,26,24.0,DET,1,1,Miran,7,Miran,1,0
68,22022,1610612741,CHI,Chicago Bulls,0022200019,2022-10-21,CHI @ WAS,L,240,100,...,15,-2.0,WAS,0,0,Vedran,7,Tata,0,1
69,22022,1610612754,IND,Indiana Pacers,0022200004,2022-10-19,IND vs. WAS,L,241,107,...,20,-7.0,WAS,1,0,Miran,9,Tata,0,1
70,22022,1610612751,BKN,Brooklyn Nets,0022200006,2022-10-19,BKN vs. NOP,L,242,108,...,25,-22.0,NOP,1,0,Vedran,4,Tata,0,1


In [17]:
# actually calculate the points!
# Doesn't address the Playoff game / Championship

bodovi = games.groupby(by=['TEAM_OWNER'])['IS_WIN'].sum()
bodovi = bodovi.sort_values(ascending=False)

print("Ovo je trenutno stanje s bodovima na daatum " + league_end_str)
print(bodovi)
# bodovi = bodovi.sort_values(by=['IS_WIN'], ascending=False)



Ovo je trenutno stanje s bodovima na daatum 10/23/2022
TEAM_OWNER
Vedran    16
Miran     10
Tata      10
Name: IS_WIN, dtype: int64


In [ ]:
 # Next steps: Play with data
# Total possible points, total games, total games vs. own teams, total ratios for all pairs...

In [18]:
print("Total number of games played:")

ukupno_tekma = games.groupby(by=['TEAM_OWNER'])['GAME_ID'].count()
ukupno_tekma = ukupno_tekma.sort_values(ascending=False)

ukupno_tekma

Total number of games played:


TEAM_OWNER
Miran     26
Vedran    24
Tata      22
Name: GAME_ID, dtype: int64

In [21]:
print("Total possible points (excl. games vs. own teams):")

potencijalno_bodovi = games.groupby(by=['TEAM_OWNER'])['IS_NOT_AGAINST_OWN'].sum()
potencijalno_bodovi  = potencijalno_bodovi.sort_values(ascending=False)

potencijalno_bodovi

Total possible points (excl. games vs. own teams):


TEAM_OWNER
Vedran    20
Tata      18
Miran     16
Name: IS_NOT_AGAINST_OWN, dtype: int64